In [9]:
from transformers import pipeline
from collections import defaultdict

# Replace this with your own checkpoint
model_checkpoint = "../models/epoch_30"
token_classifier = pipeline(
    "ner", model=model_checkpoint, aggregation_strategy="simple"
)

In [ ]:
def preprocess_description(description):
    single_description = description.strip()
    new_description = []
    last_special = -1
    for idx, letter in enumerate(single_description):
        if not (('a' <= letter and letter <= 'z') or ('A' <= letter and letter <= 'Z') or ('0' <= letter and letter <= '9') or letter == ' '):
            pretext = single_description[last_special + 1:idx].strip()
            if pretext != '' and pretext != ' ':
                new_description.append(pretext)
            new_description.append(letter.strip())
            last_special = idx
        if idx == len(single_description) - 1:
            new_description.append(
                single_description[last_special + 1:idx + 1].strip())
    return " ".join(new_description)

In [ ]:
# Replace this description
description = '''
MASSAGE ROLLER STICK: Hand massage roller with ergonomic handle design helps to ease, condition muscle soreness, reduce stiffness and sore muscles before, after workouts; training and improve blood circulation, allow your muscles to recover more quickly
ADVANCED WITH 4 WHEELS FOR MUSCLE DEEP TISSUE MASSAGE: 2 narrow/wider wheels releasing trigger points at calves, forearms, triceps, and biceps; increase blood flow and quick recovery on large muscle groups such as shins, spine, neck, hamstrings, and back
HIGH QUALITY, HIGH DURABILITY: Yes4All Muscle Roller Massage Stick is made of strong stainless steel; recycled material, highly elastic TPE, which avoids the risks of breaking or losing shape over time, help stress-crack resistant, waterproof, sweat proof
EXTRA FIRM, HIGH QUALITY: Features high density expanded polypropylene (EPP) material that is designed for deeper massage than the standard foam roller while being extremely durable, helping to boost your fitness progress for a long period of time
HIGH WEIGHT CAPACITY: EPP Foam Rollers are capable of supporting up to an amazing 300lbs of weight, enabling you to use them for a wide variety of exercises without worrying about compatibility, great for fitness enthusiasts of all levels
24-INCH LENGTH x 6-INCH ROUND: Measuring at 6”x24”, EPP Foam Rollers are very lightweight and have the perfect compactness for you to easily carry around anywhere you want and clock in solid workout sessions
'''

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        print(bullet_point)

        results = token_classifier(bullet_point)
        for res in results:
            if res['word'].lower().strip() in ['durable', 'strong', 'heavy-duty', 'heavy duty', 'stability', 'versatile']:
                continue
            group = res['entity_group']
            if res['score'] >= 0.9:
                high_score_ans[group].add(res['word'].lower().strip())
print("-"*100)            
for key in high_score_ans.keys():
    line = f"{key}: {high_score_ans[key]}"
    display(line)